<a href="https://colab.research.google.com/github/quantumhome/DataAnalysisCaseStudy/blob/master/Silver_Assignment_3_Translator_App_using_Whisper_Library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Assignment 3 craeate a traslator app which takes your voice/speech as an input(in the language you have selected, give 5 lanuagge choices)
convert it to the language you selected as output(in the output also give 5 language**

In [1]:
!pip install -U openai-whisper googletrans==4.0.0-rc1 gTTS
!apt-get install -y ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
import whisper
from googletrans import Translator
from gtts import gTTS
import os
import torch
import asyncio
from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

# --- COLAB MICROPHONE HELPER ---
RECORD_JS = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = b => new Promise(resolve => {
  const r = new FileReader()
  r.onloadend = e => resolve(e.srcElement.result)
  r.readAsDataURL(b)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record_audio(filename="mic_input.wav", seconds=5):
    print(f" Recording for {seconds} seconds...")
    display(Javascript(RECORD_JS))
    s = output.eval_js('record(%d)' % (seconds * 1000))
    b = b64decode(s.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(b)
    print("Recording finished.")
    return filename

# --- WHISPER & TRANSLATION LOGIC ---

def get_whisper_transcription(audio_path, language_code=None):
    # Detect if a GPU (CUDA) is available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    use_fp16 = True if device == "cuda" else False

    print(f" Loading Whisper model (Base) on {device.upper()}...")
    model = whisper.load_model("base", device=device)

    print(f" Analyzing audio...")
    # Passing fp16=False when on CPU removes the warning
    result = model.transcribe(audio_path, language=language_code, fp16=use_fp16)

    text = result['text'].strip()
    print(f"Recognized: {text}")
    return text

def translate_text(text, dest_language):
    """Removed 'await' as googletrans 4.0.0rc1 is synchronous"""
    translator = Translator()
    try:
        # Do not use await here
        translated = translator.translate(text, dest=dest_language)
        print(f" Translated ({dest_language}): {translated.text}")
        return translated.text
    except Exception as e:
        print(f" Translation error: {e}")
        return None

def text_to_speech(text, language_code):
    if not text: return
    tts = gTTS(text=text, lang=language_code)
    filename = "translated_audio.mp3"
    tts.save(filename)
    display(Audio(filename, autoplay=True))

# --- MAIN EXECUTION ---

language_options = {
    '1': ('English', 'en'),
    '2': ('Marathi', 'mr'),
    '3': ('Kannada', 'kn'),
    '4': ('Punjabi', 'pa'),
    '5': ('Hindi', 'hi')
}

async def main():
    while True:
        print("\n--- NEW TRANSLATION ---")
        print("1. Select Input Language (1. English, 2. Marathi, 3. Kannada, 4. Punjabi, 5. Hindi):")
        in_choice = input("Choice (or 'q' to quit): ")
        if in_choice.lower() == 'q': break
        in_lang = language_options.get(in_choice, ('English', 'en'))[1]

        print("2. Select Output Language (Target):")
        out_lang = language_options.get(input("Choice: "), ('Marathi', 'mr'))[1]

        print("\n3. Choose Source:\n(1) Microphone\n(2) Uploaded MP3 File")
        source_choice = input("Choice: ")

        audio_path = ""
        if source_choice == '1':
            duration = int(input("How many seconds to record? ") or 5)
            audio_path = record_audio(seconds=duration)
        else:
            audio_path = "/content/sample_data/AnimalSongSearch.mp3"
            if not os.path.exists(audio_path):
                print(f"File not found at {audio_path}. Please upload it to Colab.")
                continue

        # 1. Transcribe (Speech to Text)
        text = get_whisper_transcription(audio_path, language_code=in_lang)

        if text:
            # 2. Translate (Text to Text) - No await needed
            translated_text = translate_text(text, out_lang)

            # 3. Speak (Text to Speech)
            if translated_text:
                text_to_speech(translated_text, out_lang)

        input("\nPress Enter to start another translation...")

# Run the app
await main()


--- NEW TRANSLATION ---
1. Select Input Language (1. English, 2. Marathi, 3. Kannada, 4. Punjabi, 5. Hindi):
Choice (or 'q' to quit): 4
2. Select Output Language (Target):
